In [1]:
%matplotlib inline
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib import style

style.use('ggplot')

bl = pd.read_csv('~/Documents/datasets/football/D1.csv')

In [2]:
columns = [ 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
data = bl[columns]

In [59]:
# Home team win rate
n_matches = data.shape[0]

n_features = data.shape[1] - 1

n_homewins = len(data[data['FTR'] == 'H'])

win_rate = ((n_homewins) / (n_matches)) * 100

win_rate
n_matches
n_features

22

In [60]:
cat_features = ['HomeTeam','AwayTeam','FTR','HTR']
num_features = ['FTHG', 'FTAG', 'HTHG', 'HTAG',
     'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']


In [61]:
X_all = data.drop("FTR",1)
y_all = data["FTR"]

In [55]:
#Standardising the data
from sklearn.preprocessing import scale

#Center to the mean and component wise scale to unit variance
for col in num_features:
    X_all[col] = scale(X_all[col])

In [63]:
from sklearn.preprocessing import StandardScaler
#preprocessing categorical features
def preprocess_features(X):
    ''' Preprocess data and convert all categorical features to dummies'''
    #Initalise new output dataframe
    output = pd.DataFrame(index = X.index)
    
    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data is categorical,convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data,prefix = col)
        
        # Collect the revised columns
        output = output.join(col_data)
    return output

X_all = preprocess_features(X_all)
print("Preprocessed feature columns ({} total_features):\n{} ".format(len(X_all.columns), list(X_all.columns)))

Preprocessed feature columns (58 total_features):
['HomeTeam_Augsburg', 'HomeTeam_Bayern Munich', 'HomeTeam_Dortmund', 'HomeTeam_Ein Frankfurt', 'HomeTeam_FC Koln', 'HomeTeam_Freiburg', 'HomeTeam_Hamburg', 'HomeTeam_Hannover', 'HomeTeam_Hertha', 'HomeTeam_Hoffenheim', 'HomeTeam_Leverkusen', "HomeTeam_M'gladbach", 'HomeTeam_Mainz', 'HomeTeam_RB Leipzig', 'HomeTeam_Schalke 04', 'HomeTeam_Stuttgart', 'HomeTeam_Werder Bremen', 'HomeTeam_Wolfsburg', 'AwayTeam_Augsburg', 'AwayTeam_Bayern Munich', 'AwayTeam_Dortmund', 'AwayTeam_Ein Frankfurt', 'AwayTeam_FC Koln', 'AwayTeam_Freiburg', 'AwayTeam_Hamburg', 'AwayTeam_Hannover', 'AwayTeam_Hertha', 'AwayTeam_Hoffenheim', 'AwayTeam_Leverkusen', "AwayTeam_M'gladbach", 'AwayTeam_Mainz', 'AwayTeam_RB Leipzig', 'AwayTeam_Schalke 04', 'AwayTeam_Stuttgart', 'AwayTeam_Werder Bremen', 'AwayTeam_Wolfsburg', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HTR_A', 'HTR_D', 'HTR_H', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B

In [65]:
X_all = StandardScaler().fit_transform(X_all)

In [68]:
X_all = pd.DataFrame(X_all)
X_all.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,-0.242536,4.123106,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,...,0.789936,-0.421483,0.300799,-0.421667,0.139960,-0.278325,-0.271448,-0.756455,1.588575,2.261191
1,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,4.123106,-0.242536,-0.242536,-0.242536,...,0.543534,-0.788929,1.141388,1.842020,0.933068,-0.278325,-0.271448,-0.292927,-0.457218,-0.225818
2,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,4.123106,-0.242536,...,-0.442074,-0.054036,-1.380379,0.332895,-0.653148,-0.278325,-0.271448,-0.347459,-0.457218,-0.136996
3,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,4.123106,...,0.297132,0.313410,-0.119495,-0.421667,0.933068,-0.278325,-0.271448,-0.483791,0.014888,-0.009685
4,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,-0.242536,...,0.050730,2.885536,-1.380379,1.087458,2.519285,-0.278325,-0.271448,-0.347459,-0.457218,-0.107389


In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train ,y_test = train_test_split(X_all, y_all, random_state= 2,test_size = 0.25,stratify = y_all)

In [52]:
from time import time
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier ,then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # print the results
    print("Trained model in {:.4f} seconds".format(end - start))
    
def predict_labels(clf, features, target):
    '''Makes predictions using a fit classifier based on F1 score. '''
    
    # Start clock make predictions,stop clock
    start = time()
    y_pred = clf.predict(features)
    
    end  = time()
    # print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred,labels='H',average = 'macro'),sum(target == y_pred) / float(len(y_pred))

def train_predict(clf,X_train, y_train ,X_test,y_test):
    ''' Train and predict using a classifier based on F1 score'''
    
    # Indicate the classifier and the training set size
    print("Trainig a {} using a training set size of...".format(clf.__class__.__name__,))
    
    # train the classifier
    train_classifier(clf, X_train,y_train)
    
    # print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1,acc)
    print("F1 score and accuracy score for training set: {:.4f}, {:.4f}. ".format(f1,acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f}, {:.4f}. ".format(f1,acc))

In [53]:
clf_A = LogisticRegression(solver='lbfgs',random_state = 42,multi_class='auto')
clf_B = SVC(random_state = 912,kernel = 'rbf',gamma = 'scale')
clf_C = RandomForestClassifier(n_estimators = 100,max_depth = 10,random_state=82)
clf_D = xgb.XGBClassifier(seed=82)

train_predict(clf_A,X_train, y_train,X_test, y_test)
print('')

train_predict(clf_B,X_train, y_train,X_test, y_test)
print('')

train_predict(clf_C,X_train, y_train,X_test, y_test)
print('')

train_predict(clf_D,X_train, y_train,X_test, y_test)
print('')

Trainig a LogisticRegression using a training set size of...
Trained model in 0.0205 seconds
Made predictions in 0.0012 seconds.
1.0 1.0
F1 score and accuracy score for training set: 1.0000, 1.0000. 
Made predictions in 0.0008 seconds.
F1 score and accuracy score for test set: 0.9412, 0.9091. 

Trainig a SVC using a training set size of...
Trained model in 0.0083 seconds
Made predictions in 0.0046 seconds.
0.9665071770334929 0.9432314410480349
F1 score and accuracy score for training set: 0.9665, 0.9432. 
Made predictions in 0.0019 seconds.
F1 score and accuracy score for test set: 0.8485, 0.7662. 

Trainig a RandomForestClassifier using a training set size of...
Trained model in 0.0711 seconds
Made predictions in 0.0085 seconds.
1.0 1.0
F1 score and accuracy score for training set: 1.0000, 1.0000. 
Made predictions in 0.0072 seconds.
F1 score and accuracy score for test set: 0.9315, 0.8831. 

Trainig a XGBClassifier using a training set size of...
Trained model in 0.1234 seconds
Made 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

#TODO

In [128]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

'''
Load some data and do some cleaning
'''
bl = pd.read_csv('~/Documents/datasets/football/D1.csv')
columns = [ 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']
data = bl[columns]
cat_features = ['HomeTeam','AwayTeam','HTR']
num_features = ['FTHG', 'FTAG', 'HTHG', 'HTAG',
     'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A']

X_all = data.drop("FTR",1)
y_all = data["FTR"]

scaled_data = StandardScaler().fit_transform(X_all[num_features])
scaled = pd.DataFrame(scaled_data,columns = num_features)
dummies = pd.get_dummies(X_all[cat_features],prefix = [col for col, col_data in X_all[cat_features].iteritems()])

df = dummies.join(scaled)

X_train, X_test, y_train ,y_test = train_test_split(df, y_all, random_state= 0,test_size = 0.25)


/home/saazi/ml/env/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/saazi/ml/env/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [129]:
%matplotlib inline
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib import style

clf_A = LogisticRegression(solver='lbfgs',random_state = 42,multi_class='auto')
clf_B = SVC(random_state = 912,kernel = 'rbf',gamma = 'scale')
clf_C = RandomForestClassifier(n_estimators = 100,max_depth = 10,random_state=82)
clf_D = xgb.XGBClassifier(seed=82)

print('Logistic Regresssion...')
clf_A.fit(X_train, y_train)
print('Accuracy on the training subset: {:3f}'.format(clf_A.score(X_train, y_train)))
print('Accuracy on the testing subset: {:3f}'.format(clf_A.score(X_test, y_test)))
print('')

print('Support Vector Machine...')
clf_B.fit(X_train, y_train)
print('Accuracy on the training subset: {:3f}'.format(clf_B.score(X_train, y_train)))
print('Accuracy on the testing subset: {:3f}'.format(clf_B.score(X_test, y_test)))
print('')

print('Random Forest...')
clf_C.fit(X_train, y_train)
print('Accuracy on the training subset: {:3f}'.format(clf_C.score(X_train, y_train)))
print('Accuracy on the testing subset: {:3f}'.format(clf_C.score(X_test, y_test)))
print('')

print('XGBoost Classifier...')
clf_D.fit(X_train, y_train)
print('Accuracy on the training subset: {:3f}'.format(clf_D.score(X_train, y_train)))
print('Accuracy on the testing subset: {:3f}'.format(clf_D.score(X_test, y_test)))
print('')

Logistic Regresssion...
Accuracy on the training subset: 1.000000
Accuracy on the testing subset: 0.896104

Support Vector Machine...
Accuracy on the training subset: 0.925764
Accuracy on the testing subset: 0.805195

Random Forest...
Accuracy on the training subset: 1.000000
Accuracy on the testing subset: 0.792208

XGBoost Classifier...
Accuracy on the training subset: 1.000000
Accuracy on the testing subset: 0.974026

